In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import main as m
import regex as re

In [2]:
# investigator 
# ablilities with embedid values not capturing correctly
# Complex deck reqs not being captured correctly
# some must includes not being captured correctly

In [3]:
invest, assets, events, skills = m.get_urls_by_type()


Getting investigator URL's...
Getting asset URL's...
Getting event URL's...
Getting skill URL's...


In [4]:
##########################################Helper functions#########################################################
def get_soup(url):
    '''Child of get_skills_df
       Takes in a url for a card
       Returns html request result parsed using beautiful soup'''

    # create request and soup objects
    html = requests.get(url)

    soup = BeautifulSoup(html.content, 'html.parser')

    # locate urls on page and return
    return soup.find(id='list')


def get_text_for_icon(text):

    # replace icon html with matching word in all caps
    icon_types = [
                  'action',
                  'reaction',
                  'wild', 
                  'willpower', 
                  'combat', 
                  'agility', 
                  'intellect', 
                  'wild',
                  'curse', 
                  'bless',
                  'rogue',
                  'survivor',
                  'seeker',
                  'guardian',
                  'mystic',
                  'neutral',
                  'skull',
                  'tablet',
                  'cultist',
                  'elder sign']
    
    for icon in icon_types:
    
        text = text.replace(f'<span class="icon-{icon}" title="{icon.capitalize()}"></span>', 
                                  f'{icon.upper()}')
        
        text = text.replace(f'<div class="card-text border-{icon}">\n<p>', '')

    text = text.replace(f'<span class="icon-wild" title="Any Skill"></span>', 'WILD')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Sign"></span>', 'ELDER_SIGN')

    text = text.replace(f'<span class="icon-elder_sign" title="Elder Thing"></span>', 'ELDER_THING')

    text = text.replace(f'<span class="icon-lightning" title="Fast Action"></span>', 'FAST ACTION')

    text = text.replace(f'<span class="icon-auto_fail" title="Auto Fail"></span>', 'TENTACLES')

    return text

def clean_html(text):

    # delete extraneous html
    dirt = [
            '</p>\n</div>',
            '</p>',
            '<p>',
            '<b>',
            '</b>',
            '<br/',
            '<1>',
            '<i>',
            '</i>',
            '><span>']
    
    for item in dirt:
        
        text = text.replace(item,'')

    return text

In [45]:
def get_xp(results):
    ''' Child of get_card_traits
        Takes in an BS object containing card information
        Returns XP cost of card if it exists
        Otherwise returns 0'''
    
    # locate xp info in object
    xp = results.find('div', class_='card-props')
    
    # if there is no value return 0
    if xp == None:
        
        return 0
    
    # otherwise return perform minor cleaning on value and return
    else:
        
        return re.search('[0-9]', str(xp)).group()
    
    
def get_icons(results):
    '''Child of get_card_traits
       Takes in request results for an arkhamdb page containing player card data
       Returns a string represintation of skill test icons on the card'''
      
    icons = ''

    # list containing each icon type
    icon_types = ['wild', 'willpower', 'combat', 'agility', 'intellect']

    # itterate through icon types
    for stat in icon_types:

        # get number of that icon on card from request results
        num_icons = len(results.find_all('span', class_=f'icon icon-{stat} color-{stat}'))

        # add that icon name to a string for each time it appears in request results
        for icon in range(num_icons):

            icons += f'{stat} '
            
    return icons.upper()[:-1]


#####find changes to find_all#######################################################################
def get_ability(results, faction):
    '''Child of get_card_traits
        Takes in request results for an arkhamdb page containing player card data
        Returns a string represintation of skill test icons on the card'''
    
    # gets html object and convert to string
    ability_string = str(results.find_all('div', class_=f'card-text border-{faction.lower()}'))
    
    # convert html to string, replace icons in text with string represintations
    ability_text = clean_ability_string(ability_string)
    
    return ability_text

In [23]:
assets[3]

'https://arkhamdb.com/card/07305'

In [ ]:
[
    
 #title,
 #sub_title,
 #faction, 
 #tipe, 
 body_slot
 #traits,
 cost,
 #xp,
 #icons,
 #ability,
 #artist,
 #expansion,
 #flavor

]

In [36]:
results = get_soup('https://arkhamdb.com/card/09089')

In [46]:
title = results.find('a', class_='card-name card-tip').text.replace('\n', '').replace('\t', '')

sub_title = results.find('div', class_='card-subname small').text.replace('\n', '').replace('\t', '')

faction = results.find('span', class_='card-faction').text.replace('\n', '').replace('\t', '')

tipe = results.find('span', class_='card-type').text.replace('\n', '').replace('\t', '')

traits = results.find('p', class_='card-traits').text.replace('\n', '').replace('\t', '')

xp =  get_xp(results)

ability = get_ability(results, faction)

flavor = results.find('div', class_='card-flavor small').text.replace('\n', '').replace('\t', '')

artist = results.find('div', class_='card-illustrator').text.replace('\n', '').replace('\t', '')

expansion = results.find('div', class_='card-pack').text.replace('\n', '').replace('\t', '').replace('.', '')

icons = get_icons(results)

In [66]:
results = get_soup('https://arkhamdb.com/find?q=t%3Aasset&decks=player')

In [67]:
results.find('div', class_='qtip-content')

In [68]:
results.find('div', class_='qtip qtip-default card-content qtip-bootstrap qtip-thronesdb qtip-thronesdb-text qtip-pos-lc')

In [39]:
title

"Binder's Jar"

In [38]:
sub_title

'Interdimensional Prison'

In [40]:
faction

'Mystic'

In [41]:
tipe

'Asset. '

In [42]:
traits

'Item. Relic.'

In [43]:
xp

'2'

In [47]:
ability

"[You get +1 arcane slot for each enemy beneath Binder's Jar.REACTION After a non-Elite enemy is defeated at your location: Place that enemy facedown beneath Binder's Jar (limit 2 enemies beneath it).REACTION When an enemy attacks you, discard an enemy beneath Binder's Jar that shares a Trait with the attacking enemy: Cancel that attack.]"

In [48]:
flavor

''

In [49]:
artist

'Steve Hamilton'

In [50]:
expansion

'The Scarlet Keys Investigator Expansion #89'

In [51]:
icons

'WILLPOWER'